# HW06: Word Embeddings

Remember that these homework work as a completion grade. **You can <span style="color:red">not</span> skip one section this homework.**

**Essay Feedback**

Please provide feedback to two classmates' essays on Eduflow.

**Training word2vec**

In this section, we train a word2vec model using gensim. We train the model on text8 (which consists of the first 90M characters of a Wikipedia dump from 2006 and is considered one of the benchmarks for evaluating language models).

In [ ]:
import gensim.downloader as api
api.info("text8")

In [ ]:
dataset = api.load("text8")

In [ ]:
from gensim.models import Word2Vec

##TODO train a word2vec model on this dataset, only consider words which appear at least 10 times in the corpus

model = Word2Vec(dataset,
                workers = 5,
                size=300,       
                min_count = 10,  
                window = 3,      
                sample = 1e-3,
               )

model.most_similar('society')

**Word Similarities**

gensim models provide almost all the utility you might want to wish for to perform standard word similarity tasks. They are available in the .wv (wordvectors) attribute of the model, more details could be found [here](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [ ]:
word_vectors = model.wv

##TODO find the closest words to king
word_vectors.most_similar('king')

King is to man as woman is to X

In [ ]:
##TODO find the closest word for the vector "woman" + "king" - "man"
word_vectors.most_similar(positive = ['woman', 'king'], negative = ['man'])[0]

**Evaluate Word Similarities** 

One common way to evaluate word2vec models are word analogy tasks. Let's check how good our model is on one of those. We consider the [WordSim353](http://alfonseca.org/eng/research/wordsim353.html) benchmark, the task is to determine how similar two words are.

In [ ]:
!wget http://alfonseca.org/pubs/ws353simrel.tar.gz
!tar xf ws353simrel.tar.gz

path = "wordsim353_sim_rel/wordsim_similarity_goldstandard.txt"

def load_data(path):
    X, y = [], []
    with open(path) as f:
        for line in f:
            line = line.strip().split("\t")
            X.append((line[0], line[1])) # each entry in x contains two words, e.g. X[0] = (tiger, cat)
            y.append(float(line[-1])) # each entry in y is the annotation how similar two words are, e.g. Y[0] = 7.35
    return X, y

X, y = load_data(path)
print (X[:3], y[:3])

In [ ]:
##TODO compute how similar the pairs in the WordSim353 are according to our model
##TODO if  aword is not present in our model, we assign similarity 0 for the respective text pair
scores = [(model.similarity(pair[0], pair[1]), pair) if (pair[0] in model and pair[1] in model) else (0, pair) for pair in X ]

scores

In [ ]:
from scipy.stats import spearmanr

##TODO compute spearman's rank correlation between our prediction and the human annotations
sc = [item[0] for item in scores]
spearmanr(sc, y)

In [ ]:
import spacy
en = spacy.load('en_core_web_sm')

##TODO compute word similarities in the WordSim353 dataset using spaCy word embeddings
##TODO compute spearman's rank correlation between these similarities and the human annotations
# Don't worry if results are not too convincing for this experiment

spacy_scores = [(en(pair[0]).similarity(en(pair[1])), pair) for pair in X]
score = [item[0] for item in spacy_scores]
spearmanr(score, y)